In [1]:
import pickle
import pandas as pd
import glob
import json
from tqdm import tqdm
import os
import shutil
import time

ModuleNotFoundError: No module named 'pandas'

# Create jsons

In [2]:
# Load the webdata
web_data = pickle.load( open( "web_data.pkl", "rb" ) )
web_scraping = pickle.load( open( "vdab.pickle", "rb" ) )
web_scraping = {url.replace('https://www.vdab.be', 'https://www-rel1ldv.ops.vdab.be'): content for url, content in zip(web_scraping.URL, web_scraping.CONTENT) if content is not None}

# path to save to
web_path = 'data/web_json'
shutil.rmtree(web_path)
os.mkdir(web_path)

# fields to keep
keys = ['title', 'type', 'breadcrumb_field', 'url', 'description', 'content', 'link_ankers']

# write the jsons
scraped_documents  = 0
new_docs = []
for data in tqdm(web_data):
    
    # take the data from the webscraping if it exists
    if len(data['content']) < 20:
        continue
        
    # if the url is known from scraping, take those documents
    if data['url'] in web_scraping:
        scraped_documents+= 1
        
        # get the new text
        new_text = web_scraping[data['url']]
        new_text = new_text.split('- Nieuw!', 1)[1] if 'Nieuw!' in new_text else new_text
        new_text = new_text.replace('https://www.vdab.be', 'https://www-rel1ldv.ops.vdab.be')
        
        # replace the text
        data['content'] = new_text
        new_docs.append(new_text)
    

    
    with open(f"{web_path}/web_{len(glob.glob(f'{web_path}/*'))}.json", 'w') as f:
        json.dump({k:data[k] for k in keys if k in data}, f)
    
f"Scraped documents, recovered: {scraped_documents}"

FileNotFoundError: [Errno 2] No such file or directory: 'web_data.pkl'

In [253]:
# Load the data
intranet_data = pickle.load( open( "intranet_data.pkl", "rb" ) )

# Path to save to
intranet_path = 'data/intranet_json'
shutil.rmtree(intranet_path)
os.mkdir(intranet_path)

# fields to keep
keys = ['title', 'type', 'field_trefwoorden_name', 'domain_name', 'category_name', 'field_faq_summary','url', 'content', 'link_ankers']

# write the jsons
for data in tqdm(intranet_data):
    # write to file
    if 'field_faq_summary' in data:
        if len(data['field_faq_summary']) < 5:
            #remove that field
            del data['field_faq_summary']
            
    if 'description' in data:
        print('oei')
        
    if 'domain_name' in data:
        data['domain_name'] = [x for x in data['domain_name'] if len(x) >=3]
        if len(data['domain_name']) == 0:
            del data['domain_name']
            
    if 'field_trefwoorden_name' in data:
        data['field_trefwoorden_name'] = [x for x in data['field_trefwoorden_name'] if len(x) >=3]
        if len(data['field_trefwoorden_name']) == 0:
            del data['field_trefwoorden_name']
            
    with open(f"{intranet_path}/intranet_{len(glob.glob(f'{intranet_path}/*'))}.json", 'w') as f:
        json.dump({k:data[k] for k in keys if k in data}, f)

100%|██████████| 869/869 [00:00<00:00, 1070.02it/s]


# Translate

In [188]:
import requests, uuid, json

# Add your key and endpoint
key = "63d4db987dda4539bfaf119ecaee20d4"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "westeurope"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'nl',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text': 'Ik heb honger'
},{
    'text': 'wij hebben honger'
}]
response = requests.post(constructed_url, 
                         params=params,
                         headers=headers, 
                         json=body).json()
response

{'error': {'code': 403001,
  'message': 'The operation is not allowed because the subscription has exceeded its free quota.'}}

## Webdata

In [149]:
if False:   
    # Path to save to
    web_translation_path = f'data/web_translated_json'
    shutil.rmtree(web_translation_path)
    os.mkdir(web_translation_path)

In [150]:
tokens = 0
for e, file in tqdm(enumerate(glob.glob(f'{web_path}/web_*'))):
    
    # loop over the existing json files
    with open(file) as f:
        
        # load the json
        data= json.load(f)

        # fields
        fields = []
        body = []
        split = False
        
        # loop over the fields
        # !!! 'breadcrumb_field' heb ik eruit gelaten !!!
        for field in ['content', 'description', 'title']:
            
            # if there is not enough content
            if len(data.get(field, '')) < 3:
                continue
                
            # count the tokens
            tokens += len(data[field])
            
            #
            # handle the text
            #
            if len(data[field]) > 40_000:

                # split the text                     
                texts = []
                txt = ''
                # loop over all the words
                for word in data[field].split(' '):
                    # add the 
                    txt += f" {word}"
                    # txt in texts, if the length is + 40_000
                    if len(txt) > 40_000:
                        texts.append(txt.strip())
                        txt = ''
                else:
                    if len(txt) > 0:
                        texts.append(txt.strip())

                #
                # fill in the body and fields
                #
                for t_e, txt in enumerate(texts):
                    fields.append(f'{field}___{t_e}')
                    body.append({'text': txt})
                    split = True

            else:
                #
                # Normal situation
                #

                # add the field type
                fields.append(field)
                # add the text
                body.append({'text': data[field]})

                    
        #if we've stuff to translate
        if len(body) > 0:
            if split:
                print('Split')
                for field, b in zip(fields, body):

                    # get the translation
                    response = requests.post(constructed_url, 
                                                 params=params, 
                                                 headers=headers, 
                                                 json=[b]).json()
                    response_text = response[0]['translations'][0]['text']

                    # get the field
                    field_name = f"{field.split('___', 1)[0]}_translation"

                    # create an item
                    ## if fielname does not exist yet, assign response_text
                    ## otherwise add the response_text
                    data[field_name] = response_text if field_name not in data else data[field_name] + ' ' + response_text

            else:
                # do everything at once
                # translate
                response = requests.post(constructed_url, 
                                         params=params, 
                                         headers=headers, 
                                         json=body).json()
                for i in range(len(fields)):
                    data[f'{fields[i]}_translation'] = response[i]['translations'][0]['text']
                    
        
        
        # Store the data
        with open(f"{file.replace(web_path, web_translation_path)}", 'w') as f:
            json.dump({k:data[k] for k in list(data)}, f)
            
    time.sleep(3)
tokens
 

31it [01:37,  3.12s/it]

Split


612it [32:49,  3.22s/it]


1571434

# Intranet

In [276]:
if False:   
    # Path to save to
    intranet_translation_path = f'data/intranet_translated_json'
    shutil.rmtree(intranet_translation_path)
    os.mkdir(intranet_translation_path)

In [ ]:
tokens = 0
a = []
for e, file in tqdm(enumerate(glob.glob(f'{intranet_path}/intranet_*'))):
    if file.replace(intranet_path, intranet_translation_path) in glob.glob(f'{intranet_translation_path}/*'):
        continue
        
    # loop over the existing json files
    with open(file) as f:
        
        # load the json
        data= json.load(f)
        
        fields = []
        body = []
        
        chars = 0
        # loop over the fields
        keys = ['title', 'field_trefwoorden_name', 'domain_name', 'category_name', 'field_faq_summary', 'content']
        for field in keys:
    
            # if there is not enough content
            if len(data.get(field, '')) < 3:
                if field not in data:
                    continue
                else:
                    if type(data[field]) == str:
                        continue

            # count the amount of chars
            tmp_text = []
            if field == "content":
                tmp_text = [f'{q} <VDAB-SPLIT> {a}' for q, a in data[field]]
            
            elif field in ['field_trefwoorden_name', 'domain_name']:
                tmp_text = data[field]
            else:
                tmp_text = [data[field]]
                
            # create the body and fields
            for item in tmp_text:
                
                fields.append(field)
                # add the text
                body.append({'text': item})
                
        #
        # If we've all the items.
        #
        
        if len(body) > 0:

            response = requests.post(constructed_url, 
                                     params=params, 
                                     headers=headers, 
                                     json=body).json()

            # loop over each item
            for i in range(len(fields)):
                
                # get the fieldname +  the text
                field_name = f"{fields[i]}_translation"
                response_text = response[i]['translations'][0]['text']

                # if it is the content, split the text on the special key
                if fields[i] == "content":
                    if field_name not in data:
                        data[field_name] = []
                    # get the response
                    q_and_a = response_text.split('<VDAB-SPLIT>')
                    data[field_name].append((q_and_a[0].strip(), q_and_a[1].strip()))

                # if it is a list, 
                elif fields[i] in ['field_trefwoorden_name', 'domain_name']:
                    if field_name not in data:
                        data[field_name] = []
                    data[field_name].append(response_text)
                    
                # otherwise, if it is just a text
                else:
                    data[field_name] = response_text

            # Store the data
            with open(f"{file.replace(intranet_path, intranet_translation_path)}", 'w') as f:
                json.dump({k:data[k] for k in list(data)}, f)           

        time.sleep(9)
    tokens


315it [24:20,  9.33s/it]

In [303]:
len(glob.glob(f'{intranet_path}/intranet_*')), len(glob.glob(f'{intranet_translation_path}/intranet_*'))

(869, 869)

# create the final results

In [13]:
intranet_translation_path = f'data/intranet_translated_json'
web_translation_path = f'data/web_translated_json'

In [14]:
from tqdm import tqdm
import glob
import json

In [12]:
k = set([])
for e, file in tqdm(enumerate(glob.glob(f'{intranet_translation_path}/intranet_*'))):

    
    # loop over the existing json files
    with open(file) as f:
        
        # load the json
        data= json.load(f)
        

        nl_doc = ['title',
                  'field_faq_summary',
                  'content',
                  'category_name',
                  'domain_name',
                  'field_trefwoorden_name',
                  'url',
                  'link_ankers',
                  'type']

        document = None
        for field in nl_doc:

            # if field is title
            if field == 'title' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = data[field]

            elif field == 'field_faq_summary' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n {data[field]}"

            elif field == 'content' and field in data:
                document = document.strip()
                document += f" \n\n "
                for question, answer in data[field]:
                    document += f"Vraag aan de VDAB consulent: {question}? \n Antwoord of Actie: {answer} \n\n "

            elif field == 'category_name' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Categorie: {data[field]}"

            elif field == 'domain_name' and field in data:
                data[field] = list({x.lower() for x in data[field] if len(x) >= 3})
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Domein: {' - '.join(list(set(data[field])))}"

            elif field == 'field_trefwoorden_name' and field in data:
                data[field] = list({x.lower() for x in data[field] if len(x) >= 3})
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Tags: {', '.join(list(set(data[field])))}"

            elif field == 'url' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Url: {data[field]}"

            elif field == 'link_ankers' and field in data:
                data[field] = [(x,y) for x,y in data[field] if len(x) > 5]
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Volgende paginas: {' - '.join([f'{txt}, via {url}' for url, txt in data[field]])}"

            elif field == 'type' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Type: {data[field]}"

        data['document'] = document

        tr_doc = ['title_translation',
                  'field_faq_summary_translation',
                  'content_translation',
                  'category_name_translation',
                  'domain_name_translation',
                  'field_trefwoorden_name_translation',
                  'url',
                  'link_ankers',
                  'type']

        document = None
        for field in tr_doc:

            # if field is title
            if field == 'title_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = data[field]

            elif field == 'field_faq_summary_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n {data[field]}"

            elif field == 'content_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n "
                for question, answer in data[field]:
                    document += f"Question for the VDAB counselor: {question}? \n Answer or Action: {answer} \n\n "

            elif field == 'category_name_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Category: {data[field]}"

            elif field == 'domain_name_translation' and field in data:
                data[field] = list({x.lower() for x in data[field] if len(x) >= 3})
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Domain: {' - '.join(list(set(data[field])))}"

            elif field == 'field_trefwoorden_name_translation' and field in data:
                data[field] = list({x.lower() for x in data[field] if len(x) >= 3})
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Tags: {', '.join(list(set(data[field])))}"

            elif field == 'url' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Url: {data[field]}"

            elif field == 'link_ankers' and field in data:
                data[field] = [(x,y) for x,y in data[field] if len(x) > 5]
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Next pages: {' - '.join([f'{txt}, via {url}' for url, txt in data[field]])}"

            elif field == 'type' and field in data:
                document = document.strip()
                document += f" \n\n Type: {data[field]}"

        data['document_translation'] = document

        #
        #
        #
        if "field_faq_summary" in data:
            data['description'] = data["field_faq_summary"]
            data['description_translation'] = data["field_faq_summary_translation"]
            del data["field_faq_summary"]
            del data["field_faq_summary_translation"]
        
        #
        # Store the data
        #
        with open(f"{file.replace('intranet_translated_json', 'output')}", 'w') as f:
            json.dump({k:data[k] for k in list(data)}, f)           


869it [00:00, 1016.01it/s]


In [15]:
k = set([])
for e, file in tqdm(enumerate(glob.glob(f'{web_translation_path}/web_*'))):

    
    # loop over the existing json files
    with open(file) as f:
        
        # load the json
        data= json.load(f)
        

        nl_doc = ['title',
                  'breadcrumb_field',
                  'description',
                  'content',
                  'url',
                  'link_ankers',
                  'type']
        
        document = None
        for field in nl_doc:

            # if field is title
            if field == 'title' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = data[field]

            elif field == 'breadcrumb_field' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n {data[field]}"
                
            elif field == 'description' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n {data[field]}"
                
            elif field == 'content' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n {data[field]}"

            elif field == 'url' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Url: {data[field]}"

            elif field == 'link_ankers' and field in data:
                data[field] = [(x,y) for x,y in data[field] if len(x) > 5]
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Volgende paginas: {' - '.join([f'{txt}, via {url}' for url, txt in data[field]])}"

            elif field == 'type' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Type: {data[field]}"

        data['document'] = document

        tr_doc = ['title_translation',
                  'breadcrumb_field',
                  'description_translation',
                  'content_translation',
                  'url',
                  'link_ankers',
                  'type']

        document = None
        for field in tr_doc:

            # if field is title
            if field == 'title_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = data[field]

            elif field == 'breadcrumb_field' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n {data[field]}"
                
            elif field == 'description_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n {data[field]}"
                
            elif field == 'content_translation' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n {data[field]}"

            elif field == 'url' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Url: {data[field]}"

            elif field == 'link_ankers' and field in data:
                data[field] = [(x,y) for x,y in data[field] if len(x) > 5]
                if len(data[field]) == 0:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Next pages: {' - '.join([f'{txt}, via {url}' for url, txt in data[field]])}"

            elif field == 'type' and field in data:
                if len(data[field]) <= 2:
                    del data[field]
                    continue
                document = document.strip()
                document += f" \n\n Type: {data[field]}"

        data['document_translation'] = document

        
        #
        # Store the data
        #
        with open(f"{file.replace('web_translated_json', 'output')}", 'w') as f:
            json.dump({k:data[k] for k in list(data)}, f)      

612it [00:00, 1024.68it/s]


In [3]:
# loop though the folder of json files and create a csv file
import glob
import json
import pandas as pd
from tqdm import tqdm
data = []
for file in tqdm(glob.glob('data/output/intranet_*')):
    with open(file) as f:
        json_data = json.load(f)
        data.append(json_data)
df_intranet = pd.DataFrame(data)
df_intranet.head()

  0%|          | 0/869 [00:00<?, ?it/s]

100%|██████████| 869/869 [00:00<00:00, 6310.54it/s]


,title,type,field_trefwoorden_name,domain_name,category_name,url,content,link_ankers,title_translation,field_trefwoorden_name_translation,domain_name_translation,category_name_translation,content_translation,document,document_translation,description,description_translation
0,Naar waar dispatchen: Sectorale bemiddeling vs...,intranet,"[dispatchingwizard, dispatching, diwi, sectora...",[diwi],Servicelijn,https://intranet-rel1ldv.ops.vdab.be/faq/servi...,[[Verschil tussen sectorale bemiddeling en int...,[[https://intranet-rel1ldv.ops.vdab.be/faq/ser...,To where to dispatch: Sectoral mediation vs. I...,"[sectoral mediation, dispatching, diwi, idv, d...",[diwi],Service line,[[Difference between sectoral mediation and in...,Naar waar dispatchen: Sectorale bemiddeling vs...,To where to dispatch: Sectoral mediation vs. I...,NaN,NaN
1,Uitbetalingsinstelling (UI) vraagt rechtzettin...,intranet,NaN,NaN,Servicelijn,https://intranet-rel1ldv.ops.vdab.be/faq/servi...,"[[Via telefoon of e-mail, De UI vraagt enkel e...",[[https://intranet-rel1ldv.ops.vdab.be/faq/ser...,Payout setting (UI) requests correction for cu...,NaN,NaN,Service line,"[[By phone or e-mail, The UI only asks for a c...",Uitbetalingsinstelling (UI) vraagt rechtzettin...,Payout setting (UI) requests correction for cu...,Dit document geeft de werkwijze van de Allo re...,This document shows the working method of the ...
2,Sollicitatiefeedback,intranet,"[sollicitatie, verwijzing, sollicitatie opdracht]","[vacatures en sollicitaties, solliciteren]",Servicelijn,https://intranet-rel1ldv.ops.vdab.be/faq/servi...,"[[Wat is sollicitatiefeedback?, De feedback bi...",NaN,Application feedback,"[reference, application, application assignment]","[vacancies and applications, apply]",Service line,"[[What is application feedback?, The feedback ...",Sollicitatiefeedback \n\n verzamelde info omtr...,Application feedback \n\n Information collecte...,verzamelde info omtrent de sollicitatiefeedbac...,Information collected about the application fe...
3,Tab Gewenste Jobs,intranet,[mijn loopbaan],[tab profiel en gewenste jobs],Servicelijn,https://intranet-rel1ldv.ops.vdab.be/faq/servi...,"[[Gewenste jobs, Gewenste job verplicht veld g...",[[https://intranet-rel1ldv.ops.vdab.be/faq/ser...,Desired Jobs tab,[my career],[tab profile and desired jobs],Service line,"[[Desired, jobs Desired job required field des...",Tab Gewenste Jobs \n\n FAQ's voor de Serviceli...,Desired Jobs tab \n\n FAQ's for the Service Li...,FAQ's voor de Servicelijn over het mapje 'matc...,FAQ's for the Service Line about the folder 'm...
4,Mijn VDAB voor entiteiten Vlaamse Overheid,intranet,"[vlaamse overheid, mijn vdab, agentschap overh...",[welk kiezen],Servicelijn,https://intranet-rel1ldv.ops.vdab.be/faq/servi...,"[[Situering?, Meer informatie hierover in het ...",[[https://intranet-rel1ldv.ops.vdab.be/faq/ser...,My VDAB for entities Flemish Government,"[flemish government, agency government employe...",[which to choose],Service line,"[[Location?, More information about this in th...",Mijn VDAB voor entiteiten Vlaamse Overheid \n\...,My VDAB for entities Flemish Government \n\n A...,Aangepaste procedure specifiek voor de Service...,Adapted procedure specifically for the Service...


In [5]:
df_intranet.iloc[0]["content_translation"]

[['Difference between sectoral mediation and intensive services?',
  'Is the distance to the labor market. Sectoral mediation is done towards the NEC (normal economic circuit). For customers in intensive services, the distance to the NEC is - for the time being - too great.'],
 ['Where to dispatch when in doubt between sectoral and intensive?',
  'Always to a sectoral mediation team.'],
 ['Who dispatches to intensive services?',
  'Dutch-speaking customers who meet at least 1 of the following conditions: With a certificate BUSO OV2 (cat. 03, 05 or 00). With a NIHDI status without mediator in cat.32 ( not for cat.03 ) Customer with income replacement allowance FPS Social Security (cat. 06) In cat.05 who need help from VDAB. See paragraph: \' When cat. 05 dispatch? They want to work in the social economy. With a request for an activation process or a work / care trajectory. They indicate that the search for work is difficult because of: Serious health problems (age does not play a role, 

In [14]:
question_answer = " ".join(f'Question: {q} \n Answer: {a}' for q, a in df_intranet.iloc[0]["content_translation"])

In [15]:
question_answer

'Question: Difference between sectoral mediation and intensive services? \n Answer: Is the distance to the labor market. Sectoral mediation is done towards the NEC (normal economic circuit). For customers in intensive services, the distance to the NEC is - for the time being - too great. Question: Where to dispatch when in doubt between sectoral and intensive? \n Answer: Always to a sectoral mediation team. Question: Who dispatches to intensive services? \n Answer: Dutch-speaking customers who meet at least 1 of the following conditions: With a certificate BUSO OV2 (cat. 03, 05 or 00). With a NIHDI status without mediator in cat.32 ( not for cat.03 ) Customer with income replacement allowance FPS Social Security (cat. 06) In cat.05 who need help from VDAB. See paragraph: \' When cat. 05 dispatch? They want to work in the social economy. With a request for an activation process or a work / care trajectory. They indicate that the search for work is difficult because of: Serious health pr

In [8]:
data_web = []
for file in tqdm(glob.glob('data/output/web_*')):
    with open(file) as f:
        json_data = json.load(f)
        data_web.append(json_data)
df_web = pd.DataFrame(data_web)
df_web.head()

  0%|          | 0/612 [00:00<?, ?it/s]

100%|██████████| 612/612 [00:00<00:00, 2320.54it/s]


,title,type,breadcrumb_field,url,description,content,content_translation,description_translation,title_translation,document,document_translation,link_ankers
0,Wil je helpen bij het oogsten?,web,Werkzoekenden > Helpen bij het oogsten,https://www-rel1ldv.ops.vdab.be/seizoenarbeid/...,Heel wat bedrijven uit de land- en tuinbouw zo...,\n- Help onze land- en tuinbouwers! (https://w...,\n- Help our farmers and horticulturists! (htt...,Many companies from agriculture and horticultu...,Do you want to help with harvesting?,Wil je helpen bij het oogsten? \n Werkzoekende...,Do you want to help with harvesting? \n Werkzo...,NaN
1,Leer online: onbeperkt en overal,web,Werkzoekenden > Leer online,https://www-rel1ldv.ops.vdab.be/onlineleren,Bereid je voor op de arbeidsmarkt van morgen e...,\n- Opleidingen (https://www-rel1ldv.ops.vdab....,\n- Education (https://www-rel1ldv.ops.vdab.be...,Prepare for the job market of tomorrow and ref...,Learn online: unlimited and anywhere,Leer online: onbeperkt en overal \n Werkzoeken...,Learn online: unlimited and anywhere \n Werkzo...,"[[https://www-rel1ldv.ops.vdab.be/suggesties, ..."
2,"Voorwaarden terugbetaling arbeidsgereedschap ,...",web,Werkzoekenden > Voorwaarden,https://www-rel1ldv.ops.vdab.be/znarbeidsgeree...,We leggen uit aan welke voorwaarden je moet be...,We leggen uit aan welke voorwaarden je moet be...,We explain which conditions you must meet in o...,We explain which conditions you must meet in o...,"Conditions for reimbursement of work tools, cl...","Voorwaarden terugbetaling arbeidsgereedschap ,...","Conditions for reimbursement of work tools, cl...",[[https://www-rel1ldv.ops.vdab.be/arbeidshandi...
3,Wonen en werken in Brazilie,web,Werkzoekenden > Brazilie,https://www-rel1ldv.ops.vdab.be/wonen-en-werke...,NaN,Wannes in Pocos de Caldas Wat een uitdaging. E...,Wannes in Pocos de Caldas What a challenge. Ge...,NaN,Living and working in Brazil,Wonen en werken in Brazilie \n Werkzoekenden >...,Living and working in Brazil \n Werkzoekenden ...,[[https://www-rel1ldv.ops.vdab.be/werken-in-he...
4,Tewerkstellingsondersteunende maatregelen voor...,web,Werkzoekenden > Maatregelen gezondheidsproblee...,https://www-rel1ldv.ops.vdab.be/arbeidshandica...,Heb je een gezondheidsprobleem of arbeidsbeper...,\n- Gezondheidsprobleem (https://www-rel1ldv.o...,\n- Health problem (https://www-rel1ldv.ops.vd...,Do you have a health problem or work disabilit...,Employment support measures for those who have...,Tewerkstellingsondersteunende maatregelen voor...,Employment support measures for those who have...,"[[https://www-rel1ldv.ops.vdab.be/suggesties, ..."


In [20]:
df_web.to_csv('data/web_cleaned.csv', index=False)

In [21]:
df_intranet.to_csv('data/intranet_cleaned.csv', index=False)

In [13]:
df_web.iloc[0]["content_translation"]

'\n- Help our farmers and horticulturists! (https://www-rel1ldv.ops.vdab.be/helpdeoogst)\n Do you want to help with harvesting?\n- Many companies from agriculture and horticulture are looking for helping hands. Are you temporarily unemployed, a student, looking for a job...? Jump in and help an employer. This way you contribute, gain extra experience and earn a penny.\n- Help the harvest in your neighborhood\n-- Discover all vacancies\n- What does a temporary job in the harvest entail?\n-- Can you work carefully (asparagus, strawberries... are delicate products), are you independent and flexible? Then you can do your bit. Not only picking, but also sorting and packing can be part of your tasks.\n-- Keep in mind that the job can be physically demanding and that your working hours vary depending on the cultivation and weather conditions.\n- ©Shutterstock\n- Who can help with the harvest?\n-- In many situations you can do (https://www-rel1ldv.ops.vdab.be/seizoenarbeid/contract-en-loon#wie

In [109]:
df_web.shape

(612, 12)

In [110]:
df_intranet.shape

(869, 17)

In [111]:
print(df_web.shape[0] + df_intranet.shape[0])

1481


In [7]:
# metadata: url, type, 
# content: title, content

In [22]:
from typing import Any
from pathlib import Path
from typing import Sequence
import csv

from langchain.document_loaders.base import BaseLoader
from langchain.schema.document import Document
from langchain.retrievers import ParentDocumentRetriever
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [30]:
DB_PATH = Path.cwd().parent / "rag/db"
DATA_PATH = Path.cwd().parent / "rag/data"

In [50]:
class WebLoader(BaseLoader):
    def __init__(
        self,
        file_path: Path,
        text_columns: list[str],
        columns: Sequence[str] | None = None,
        csv_args: dict[str, Any] | None = None,
    ):
        """Custom loader for the VDAB web dataset.

        Args:
        ----
        file_path
            Path to the csv file.
        text_column
            Name of the column containing the text.
        columns
            Additional columns to include in the dataset. If None, no additional columns are loaded.
        csv_args
            Arguments to pass to csv.DictReader.
        """
        self.file_path = file_path
        self.text_columns = text_columns
        self.columns = columns
        self.csv_args = csv_args

    def load(self) -> list[Document]:
        """Load data into document objects."""
        csv.field_size_limit(10485760)

        docs = []
        with open(self.file_path, newline="") as file:
            csv_reader = csv.DictReader(file, **self.csv_args)
            for row in csv_reader:
                content = "\n".join(
                    f'{col.replace("_translation", "")} of the document: {row[col]}'.replace("\n", "")
                    for col in self.text_columns
                )
                try:
                    metadata = {k: v for k, v in row.items() if k in self.columns}
                except KeyError as e:
                    raise ValueError(
                        f"Columns {self.columns} not found in file {self.file_path}."
                    ) from e
                docs.append(Document(page_content=content, metadata=metadata))
        return docs


In [69]:
import pickle

In [81]:
import ast

class IntranetLoader(BaseLoader):
    def __init__(
        self,
        file_path: Path,
        text_columns: list[str],
        columns: Sequence[str] | None = None,
        csv_args: dict[str, Any] | None = None,
    ):
        """Custom loader for the VDAB web dataset.

        Args:
        ----
        file_path
            Path to the csv file.
        text_column
            Name of the column containing the text.
        columns
            Additional columns to include in the dataset. If None, no additional columns are loaded.
        csv_args
            Arguments to pass to csv.DictReader.
        """
        self.file_path = file_path
        self.text_columns = text_columns
        self.columns = columns
        self.csv_args = csv_args

    def load(self) -> list[Document]:
        """Load data into document objects."""
        csv.field_size_limit(10485760)

        docs = []
        with open(self.file_path, newline="") as file:
            csv_reader = csv.DictReader(file, **self.csv_args)
            for row in csv_reader:
                content_strings = []
                for col in self.text_columns:
                    if col == "content_translation":
                        try:
                            question_answer = " ".join(f'Question: {q} \n Answer: {a}' for q, a in ast.literal_eval(row[col]))
                            content_strings.append(f"Question answer pairs of the document: {question_answer}")
                        except:
                            print("failed")
                    else:
                        content = f'{col.replace("_translation", "")} of the document: {row[col]}'.replace("\n", "")
                        content_strings.append(content)
                content = "\n".join(content_strings)         
                try:
                    metadata = {k: v for k, v in row.items() if k in self.columns}
                except KeyError as e:
                    raise ValueError(
                        f"Columns {self.columns} not found in file {self.file_path}."
                    ) from e
                docs.append(Document(page_content=content, metadata=metadata))
        return docs


In [82]:
class VDABRetrieverClient:
    def __init__(self, chunk_size: int | None = 300, chunk_overlap: int | None = 0):
        self.documents = self._load_documents()
        self.splitter = self._load_splitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )
        self.vectorstore = None
        self.documentstore = None

    def load_retriever(
        self,
        search_type: str | None = "similarity_score_threshold",
        threshold: float | None = 0.3,
        filter: dict[str, Any] | None = {},
        k: int | None = 6,
    ) -> ParentDocumentRetriever:
        with open(DB_PATH / "store.pickle", "rb") as f:
            documentstore = pickle.load(f)
        vectorstore = Chroma(
            collection_name="full_documents",
            embedding_function=OpenAIEmbeddings(),
            persist_directory=f"{DB_PATH}/chroma_db",
        )
        return ParentDocumentRetriever(
            vectorstore=vectorstore,
            docstore=documentstore,
            child_splitter=self.splitter,
            search_kwargs={
                "search_type": search_type,
                "threshold": threshold,
                "filter": filter or {},
                "k": k,
            },
        )

    def build_retriever(
        self,
        search_type: str | None = "similarity_score_threshold",
        threshold: float | None = 0.3,
        filter: dict[str, Any] | None = {},
        k: int | None = 6,
    ) -> None:
        vectorstore = Chroma(
            collection_name="full_documents",
            embedding_function=OpenAIEmbeddings(),
            persist_directory=f"{DB_PATH}/chroma_db",
        )
        documentstore = InMemoryStore()
        retriever = ParentDocumentRetriever(
            vectorstore=vectorstore,
            docstore=documentstore,
            child_splitter=self.splitter,
            search_kwargs={
                "search_type": search_type,
                "threshold": threshold,
                "filter": filter or {},
                "k": k,
            },
        )
        retriever.add_documents(self.documents, ids=None)

        # persist document store
        with open(DB_PATH / "store.pickle", "wb") as f:
            pickle.dump(documentstore, f)

    def _load_vectorstore(self):
        vectorstore = Chroma(
            collection_name="full_documents",
            embedding_function=OpenAIEmbeddings(),
            persist_directory=f"{DB_PATH}/chroma_db",
        )
        self.vectorstore = vectorstore

    def _load_documentstore(self):
        # self.documentstore = InMemoryStore()
        with open(DB_PATH / "store.pickle", "rb") as f:
            self.documentstore = pickle.load(f)

    def _load_documents(self):
        bac_loader = WebLoader(
            file_path=DATA_PATH / "web_cleaned.csv",
            text_columns=["title_translation", "content_translation"],
            columns=["url"],
            csv_args={"quoting": csv.QUOTE_ALL},
        )
        shops_loader = IntranetLoader(
            file_path=DATA_PATH / "intranet_cleaned.csv",
            text_columns=[
                "title_translation",
                "content_translation",
            ],
            columns=["url"],
            csv_args={"quoting": csv.QUOTE_ALL},
        )
        documents_shops = shops_loader.load()
        documents_website = bac_loader.load()
        docs = documents_website + documents_shops
        return docs

    def _load_splitter(self, chunk_size: int, chunk_overlap: int):
        return RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )

In [83]:
retriever_client = VDABRetrieverClient(
    chunk_size=300, chunk_overlap=0
)

failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


In [84]:
docs = retriever_client.documents
len(docs)

1481

In [85]:
import os
with open("../openai.key", "rb") as f:
    content = f.read().decode("utf-8")
    os.environ["OPENAI_API_KEY"] = content.split("=")[-1].strip()

In [86]:
retriever_client.build_retriever()

In [87]:
retriever = retriever_client.load_retriever()

In [88]:
documents = retriever.get_relevant_documents("Wat is een IBO?")

In [89]:
for doc in documents:
    print(doc.metadata["url"])
    print(doc.page_content)

https://intranet-rel1ldv.ops.vdab.be/faq/servicelijn/ibo-aanvragen-en-starten-als-werkgever
title of the document: Applying for IBO and starting as an employer
Question answer pairs of the document: Question: Where does an employer apply for an IBO 
 Answer: Via the online tool "IBO management", which is part of My VDAB. see Form your ideal employee with an IBO (vdab.be) - video under How to apply? > 1. Request via My VDAB Manual IBO tool for employers (drive) Background info To be able to use this tool, the employer must have or apply for a My VDAB. Question: The employer does not get into the IBO tool 
 Answer: , see Wg does not see IBO management in My VDAB? Question: The employer wants to adjust the IBO application 
 Answer: Fields "Assigned to" and "Succeeded by" are blank Procedure Service line Do the adjustment. Fields "Assigned to" and/or "Succeeded by" are filled in Procedure Service Line Refer the employer to the IBO mediator mentioned in the tool. Question: The employer has 

# Demo

In [91]:
# Prompt 
# https://smith.langchain.com/hub/rlm/rag-prompt

from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")

In [92]:
# LLM

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [93]:
# RAG chain 

from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

In [101]:
rag_chain.invoke("Which data is necessary for registration? I have an adress abroad.")

AIMessage(content='To register as a cross-border worker living abroad, you need to register with category 92. The procedure includes booking a meeting on location and noting in the Plan of Action that the employee is appealing against the health insurance but wants guidance from VDAB. If you do not want guidance, a dispatch is not necessary.')

# Retriever demo

In [108]:
docs = retriever.get_relevant_documents("How should I register as a job seeker?")
for doc in docs:
    print(doc.page_content)
    print(doc.metadata["url"])

title of the document: 55+: General info
Question answer pairs of the document: Question: How to register 55+-er? 
 Answer: This depends on the situation of wz: Unemployment benefit Still in notice Outplacement ... Question: When does the type of availability appear in the file? 
 Answer: A day later via the data stream of RVA. Exception For SWT-ers (Unemployment System with Company Allowance) it can take up to a maximum of 3 months. Question: Where is the availability in the file? 
 Answer: In the banner at the top of the customer file. Note: If Actively available, nothing will appear in the banner. Question: In which files does the availability not appear 1 day after registration? 
 Answer: In files of SWT employees. Here, the availability will appear in the file at the latest 2 to 3 months after registration. Why? Job seekers apply for SWT via the payout institution. The payment institution submits the application to the RVA. RVA approves or disapproves of this. Only then can the de